# MSD - Accumulations
## Info
* Total in RnD ist Gesammttotal des PDFs!

In [1]:
import sys
sys.path.insert(0, '../../../lib/')

import tabula
import numpy as np
import pandas as pd
import importlib
import pikepdf

import pdfexport
importlib.reload(pdfexport)

from pdfexport import *

## Unlock PDF
Dieser Teil entfernt den Passwortschutz, damit das PDF in HTML exportiert werden kann

In [2]:
pdf = pikepdf.open('2019_Final Disclosure Report.pdf')
pdf.save('unlocked.pdf')

## Import PDF

In [3]:
# Read pdf into DataFrame
df_list = tabula.read_pdf("unlocked.pdf", pages='all', lattice=True, multiple_tables=True)
df = pd.concat(df_list)

## Format Table

In [4]:
df_export = df.copy()

#Drop Columns
df_export = df_export.drop(df.columns[9:],axis=1)
df_export = df_export.drop(df.columns[7],axis=1)

#Rename
df_export.columns = fix_columns_accumulations[0: 8]

#replace N/A
df_export = df_export.replace("N/A", "")

# Select
amount = df_export[df_export.type.str.contains('Aggregate amount ', na=False)]
recipients = df_export[df_export.type.str.contains('Number of Recipients', na=False)]
rnd = df_export[df_export.type.str.contains('Transfers of Value re', na=False)]

df_export = pd.concat([amount, recipients, rnd])

#Numberize
df_export = cleanup_number(df_export)
df_export = remove_dots(df_export)
df_export = replace_comma_to_dot(df_export)
df_export = amounts_to_number(df_export)

#Rename first column
df_export = df_export.reset_index(drop=True)
df_export.iloc[0,0] = "hcp_amount"
df_export.iloc[1,0] = "hco_amount"
df_export.iloc[2,0] = "hcp_count"
df_export.iloc[3,0] = "hco_count"
df_export.iloc[4,0] = "rnd"

df_export.loc[df_export.type == 'rnd', "total"] = df_export.loc[df_export.type == 'rnd', "donations_grants"]
df_export.loc[df_export.type == 'rnd', "sponsorship"] = np.NaN
df_export.loc[df_export.type == 'rnd', "donations_grants"] = np.NaN

export_acumulations(df_export, 'msd')

saved


In [ ]:
#write_to_csv(df_export, 'tmp.csv')
#write_to_excel(df_export, 'tmp.xlsx')